In [ ]:
!pip install numpy requests nlpaug
!pip install torch>=1.6.0 transformers>=4.0.0

     |████████████████████████████████| 399kB 19.2MB/s 


In [ ]:
import pandas as pd
import nlpaug.flow as naf
import nlpaug.augmenter.word as naw
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/sentiment data/train_sent_wt_mod.csv')
df_vpos = df[df['final_sent_class']=='VPos']
# df_vneg = df[df['final_sent_class']=='VNeg']
df_vpos_50k = df_vpos.sample(n=50000,random_state=1).reset_index(drop=True)
df_vpos_samp_25k = df_vpos_50k.sample(n=25000,random_state=1).reset_index(drop=True)

In [ ]:
aug = naf.Sometimes([naw.ContextualWordEmbsAug(model_path='bert-base-uncased',action='substitute',device='cuda'),
               naw.ContextualWordEmbsAug(model_path='roberta-base',action='substitute',device='cuda'),
               naw.ContextualWordEmbsAug(model_path='distilbert-base-uncased',action='substitute',device='cuda')
              #  naw.SynonymAug()
])

# aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased',action='substitute')

In [ ]:
id = df_vpos_samp_25k.iloc[0]['u_id']
ids = [id]
tx = df_vpos_samp_25k.iloc[0]['preprocessed']
l = aug.augment(tx,n=4)
for j in range(len(l)):
  ids.append(id+'_GEN_'+str(j+1))

l.insert(0,tx)
d = pd.DataFrame({'id':ids,'text':l})

In [ ]:
import time
start_time = time.time()
for i in range(1,len(df_vpos_samp_25k)):
  id = df_vpos_samp_25k.iloc[i]['u_id']
  ids = [id]
  tx = df_vpos_samp_25k.iloc[i]['preprocessed']
  l = aug.augment(tx,n=4)
  
  for j in range(len(l)):
    ids.append(id+'_GEN_'+str(j+1))

  l.insert(0,tx)
  d1 = pd.DataFrame({'id':ids,'text':l})
  d = d.append(d1,ignore_index=True)
  end_time = time.time() - start_time
  if end_time >= 300:
    print("Samples: "+str(i))
    start_time = time.time() 

Samples: 1128
Samples: 2216
Samples: 3318
Samples: 4435
Samples: 5546
Samples: 6689
Samples: 7836
Samples: 8988
Samples: 10084
Samples: 11195
Samples: 12286
Samples: 13380
Samples: 14486
Samples: 15577
Samples: 16618
Samples: 17692
Samples: 18785
Samples: 19872
Samples: 20963
Samples: 22067
Samples: 23148
Samples: 24259


In [ ]:
d['final_sent_class'] = 'VPos'

In [ ]:
len(d['text'].drop_duplicates())

123760

In [ ]:
d.to_csv('/content/drive/MyDrive/sentiment data/vpos_oversampled.csv',index=False,header=True)

In [ ]:
import pickle
with open('/content/drive/MyDrive/sentiment data/vpos_gen.pickle','wb') as f:
  pickle.dump(res,f)

In [ ]:
df.columns

Index(['u_id', 'preprocessed', 'final_sent_class'], dtype='object')

In [ ]:
aug_tx = aug.augment(tx,n=4)

In [ ]:
aug_tx

['life is no wonderful place to take',
 'life is a comfortable place you be',
 'life in really wonderful place to be',
 'there is also wonderful place too sleep']